In [1]:
import time
from pathlib import Path
from datetime import datetime
import gymnasium as gym
import json
import numpy as np
import torch
from torch import nn, optim
from torch.distributions import Normal
from torch.nn.functional import mse_loss
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm import tqdm

In [2]:
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(800, 600))
# display.start()

In [2]:
class Args:
    pass

args = Args()
args.env_id = "HalfCheetah-v4"
args.total_timesteps = 10_000_000
args.num_envs = 16
args.num_steps = 5
args.learning_rate = 5e-4
args.actor_layers = [64, 64]
args.critic_layers  = [64, 64]
args.gamma = 0.99
args.gae = 1.0
args.value_coef = 0.5
args.entropy_coef = 0.01
args.clip_grad_norm = 0.5
args.seed = 0

args.batch_size = int(args.num_envs * args.num_steps)
args.num_updates = int(args.total_timesteps // args.batch_size)

In [3]:
def make_env(env_id, capture_video=False, run_dir="."):
    def thunk():
        if capture_video:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(
                env=env,
                video_folder=f"{run_dir}/videos",
                episode_trigger=lambda x: x,
                disable_logger=True,
            )
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env = gym.wrappers.FlattenObservation(env)
        env = gym.wrappers.ClipAction(env)
        env = gym.wrappers.NormalizeObservation(env)
        env = gym.wrappers.TransformObservation(env, lambda state: np.clip(state, -10, 10))
        env = gym.wrappers.NormalizeReward(env)
        env = gym.wrappers.TransformReward(env, lambda reward: np.clip(reward, -10, 10))

        return env

    return thunk

In [4]:
def compute_advantages(rewards, flags, values, last_value, args):
    advantages = torch.zeros((args.num_steps, args.num_envs))
    adv = torch.zeros(args.num_envs)

    for i in reversed(range(args.num_steps)):
        returns = rewards[i] + args.gamma * flags[i] * last_value
        delta = returns - values[i]

        adv = delta + args.gamma * args.gae * flags[i] * adv
        advantages[i] = adv

        last_value = values[i]

    return advantages

In [5]:
class RolloutBuffer:
    def __init__(self, num_steps, num_envs, observation_shape, action_shape):
        self.states = np.zeros((num_steps, num_envs, *observation_shape), dtype=np.float32)
        self.actions = np.zeros((num_steps, num_envs, *action_shape), dtype=np.float32)
        self.rewards = np.zeros((num_steps, num_envs), dtype=np.float32)
        self.flags = np.zeros((num_steps, num_envs), dtype=np.float32)
        self.values = np.zeros((num_steps, num_envs), dtype=np.float32)

        self.step = 0
        self.num_steps = num_steps

    def push(self, state, action, reward, flag, value):
        self.states[self.step] = state
        self.actions[self.step] = action
        self.rewards[self.step] = reward
        self.flags[self.step] = flag
        self.values[self.step] = value

        self.step = (self.step + 1) % self.num_steps

    def get(self):
        return (
            torch.from_numpy(self.states),
            torch.from_numpy(self.actions),
            torch.from_numpy(self.rewards),
            torch.from_numpy(self.flags),
            torch.from_numpy(self.values),
        )

In [6]:
class ActorCriticNet(nn.Module):
    def __init__(self, observation_shape, action_dim, actor_layers, critic_layers):
        super().__init__()

        self.actor_net = self._build_net(observation_shape, actor_layers)
        self.critic_net = self._build_net(observation_shape, critic_layers)

        self.actor_net.append(self._build_linear(actor_layers[-1], action_dim, std=0.01))
        self.actor_logstd = nn.Parameter(torch.zeros(1, action_dim))

        self.critic_net.append(self._build_linear(critic_layers[-1], 1, std=1.0))

    def _build_linear(self, in_size, out_size, apply_init=True, std=np.sqrt(2), bias_const=0.0):
        layer = nn.Linear(in_size, out_size)

        if apply_init:
            torch.nn.init.orthogonal_(layer.weight, std)
            torch.nn.init.constant_(layer.bias, bias_const)

        return layer

    def _build_net(self, observation_shape, hidden_layers):
        layers = nn.Sequential()
        in_size = np.prod(observation_shape)

        for out_size in hidden_layers:
            layers.append(self._build_linear(in_size, out_size))
            layers.append(nn.Tanh())
            in_size = out_size

        return layers

    def forward(self, state):
        mean = self.actor_net(state)
        std = self.actor_logstd
        std = std.expand_as(mean).exp()
        distribution = Normal(mean, std)

        action = distribution.sample()

        value = self.critic_net(state).squeeze(-1)

        return action, value

    def evaluate(self, states, actions):
        mean = self.actor_net(states)
        std = self.actor_logstd.expand_as(mean).exp()
        distribution = Normal(mean, std)

        log_probs = distribution.log_prob(actions).sum(-1)
        entropy = distribution.entropy().sum(-1)

        values = self.critic_net(states).squeeze(-1)

        return log_probs, values, entropy

    def critic(self, state):
        return self.critic_net(state).squeeze(-1)

In [7]:
def train(args, run_name, run_dir):
    # Create tensorboard writer and save hyperparameters
    writer = SummaryWriter(run_dir)
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

    # Create vectorized environment(s)
    envs = gym.vector.AsyncVectorEnv([make_env(args.env_id) for _ in range(args.num_envs)])

    # Metadata about the environment
    observation_shape = envs.single_observation_space.shape
    action_shape = envs.single_action_space.shape
    action_dim = np.prod(action_shape)

    # Set seed for reproducibility
    if args.seed:
        torch.manual_seed(args.seed)
        state, _ = envs.reset(seed=args.seed)
    else:
        state, _ = envs.reset()

    # Create policy network and optimizer
    policy = ActorCriticNet(observation_shape, action_dim, args.actor_layers, args.critic_layers)
    optimizer = optim.RMSprop(policy.parameters(), lr=args.learning_rate, alpha=0.99, eps=1e-5)

    # Create buffers
    rollout_buffer = RolloutBuffer(args.num_steps, args.num_envs, observation_shape, action_shape)

    # Remove unnecessary variables
    del action_dim

    global_step = 0
    log_episodic_returns, log_episodic_lengths = [], []
    start_time = time.process_time()

    # Main loop
    for iter in tqdm(range(args.num_updates)):
        for _ in range(args.num_steps):
            # Update global step
            global_step += 1 * args.num_envs

            with torch.no_grad():
                # Get action
                action, value = policy(torch.from_numpy(state).float())

            # Perform action
            action = action.cpu().numpy()
            next_state, reward, terminated, truncated, infos = envs.step(action)

            # Store transition
            flag = 1.0 - np.logical_or(terminated, truncated)
            value = value.cpu().numpy()
            rollout_buffer.push(state, action, reward, flag, value)

            state = next_state

            if "final_info" not in infos:
                continue

            # Log episodic return and length
            for info in infos["final_info"]:
                if info is None:
                    continue

                log_episodic_returns.append(info["episode"]["r"])
                log_episodic_lengths.append(info["episode"]["l"])
                writer.add_scalar("rollout/episodic_return", np.mean(log_episodic_returns[-5:]), global_step)
                writer.add_scalar("rollout/episodic_length", np.mean(log_episodic_lengths[-5:]), global_step)

        # Get transition batch
        states, actions, rewards, flags, values = rollout_buffer.get()

        with torch.no_grad():
            last_value = policy.critic(torch.from_numpy(next_state).float())

        # Calculate advantages and TD target
        advantages = compute_advantages(rewards, flags, values, last_value, args)
        td_target = advantages + values

        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # Flatten batch
        states = states.reshape(-1, *observation_shape)
        actions = actions.reshape(-1, *action_shape)
        td_target = td_target.reshape(-1)
        advantages = advantages.reshape(-1)

        # Compute losses
        log_probs, td_predict, entropy = policy.evaluate(states, actions)

        actor_loss = (-log_probs * advantages).mean()
        critic_loss = mse_loss(td_target, td_predict)
        entropy_loss = entropy.mean()

        loss = actor_loss + critic_loss * args.value_coef - entropy_loss * args.entropy_coef

        # Update policy network
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(policy.parameters(), args.clip_grad_norm)
        optimizer.step()

        # Log training metrics
        writer.add_scalar("rollout/SPS", int(global_step / (time.process_time() - start_time)), global_step)
        writer.add_scalar("train/loss", loss, global_step)
        writer.add_scalar("train/actor_loss", actor_loss, global_step)
        writer.add_scalar("train/critic_loss", critic_loss, global_step)

        if iter % 1_000 == 0:
            torch.save(policy.state_dict(), f"{run_dir}/policy.pt")

    # Save final policy
    torch.save(policy.state_dict(), f"{run_dir}/policy.pt")
    print(f"Saved policy to {run_dir}/policy.pt")

    # Close the environment
    envs.close()
    writer.close()

    # Average of episodic returns (for the last 5% of the training)
    indexes = int(len(log_episodic_returns) * 0.05)
    mean_train_return = np.mean(log_episodic_returns[-indexes:])
    writer.add_scalar("rollout/mean_train_return", mean_train_return, global_step)

    return mean_train_return

In [8]:
def eval_and_render(args, run_dir):
    # Create environment
    env = gym.vector.SyncVectorEnv([make_env(args.env_id, capture_video=True, run_dir=run_dir)])

    # Metadata about the environment
    observation_shape = env.single_observation_space.shape
    action_shape = env.single_action_space.shape
    action_dim = np.prod(action_shape)

    # Load policy
    policy = ActorCriticNet(observation_shape, action_dim, args.actor_layers, args.critic_layers)
    filename = f"{run_dir}/policy.pt"
    print(f"reading {filename}...")
    policy.load_state_dict(torch.load(filename))
    policy.eval()

    count_episodes = 0
    list_rewards = []

    state, _ = env.reset()

    # Run episodes
    while count_episodes < 30:
        with torch.no_grad():
            action, _ = policy(torch.from_numpy(state).float())

        action = action.cpu().numpy()
        state, _, _, _, infos = env.step(action)

        if "final_info" in infos:
            info = infos["final_info"][0]
            returns = info["episode"]["r"][0]
            count_episodes += 1
            list_rewards.append(returns)
            print(f"-> Episode {count_episodes}: {returns} returns")

    env.close()

    return np.mean(list_rewards)

In [9]:
# Create run directory
run_time = str(datetime.now()).replace(" ", "_" ).replace(":", "-").split(".")[0].replace("-", "_")
print(f"run time: {run_time}")
run_name = "A2C_PyTorch"

run_dir = Path(f"runs/{run_name}/{run_time}")
run_dir.mkdir(parents=True, exist_ok=True)

with open(run_dir / "args.json", "w") as fp:
    json.dump(args.__dict__, fp)

run time: 2024_01_25_14_30_58


In [10]:
print(f"Commencing training of {run_name} on {args.env_id} for {args.total_timesteps} timesteps.")
print(f"Results will be saved to: {run_dir}")
mean_train_return = train(args=args, run_name=run_name, run_dir=run_dir)
print(f"Training - Mean returns achieved: {mean_train_return}.")

Commencing training of A2C_PyTorch on HalfCheetah-v4 for 10000000 timesteps.
Results will be saved to: runs\A2C_PyTorch\2024_01_25_14_30_58


  0%|          | 0/125000 [00:00<?, ?it/s]

torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([5, 16, 17]) torch.Size([5, 16, 6]) torch.Size([5, 16]) torch.Size([5, 16]) torch.Size([5, 16])
tensor([ -9.5079,  -6.9118,  -8.7038,  -7.7109,  -7.1455,  -7.3211, -11.3247,
         -9.4816, -10.7889,  -7.1531,  -9.8028, -11.1268,  -7.5115, -12.2696,
        -10.4919,  -7.2353, -11.5606,  -9.6581, -10.4265,  -8.8145,  -6.2998,
         -9.3900,  -8.9101,  -7.9214,  -9.1861,  -9.5402,  -7.4585,  -9.2766,
        -13.0857,  -8.0035,  -7.2780,  -7.4284, -10.6952,  -8.6997,  -8.4272,
         -7.3089,  -8.4338,  -5.9591,  -8.2958,  -6.7976,  -7.7187,  -7.7464,
         -8.6061,  -8.5287,  -7.0908,  -6.9176, -11.4913, -10.0649,  -7.4291,
         -7.8363,  -8.4858,  -8.2495,  -7.2911,  -6.3886,  -9.3578,  -9.3288,
        -13.1002,  -6.2564,  -7.4353,  -6.7484,  -8.6085,  -7.5471,  -7.85

  0%|          | 4/125000 [00:00<4:05:15,  8.49it/s] 

torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([5, 16, 17]) torch.Size([5, 16, 6]) torch.Size([5, 16]) torch.Size([5, 16]) torch.Size([5, 16])
tensor([ -7.7544,  -8.5907,  -7.6455, -12.2562,  -7.0277,  -7.2115,  -8.3066,
         -9.3771,  -6.9276, -13.7222,  -7.6208,  -7.9477,  -6.0989,  -8.6889,
         -8.8624, -11.2606,  -5.8080,  -8.1976,  -7.9613,  -8.0889, -11.4663,
         -6.5733,  -9.2791,  -7.2414,  -7.4477, -11.8833, -11.7853,  -7.8465,
         -8.6765,  -6.7869,  -7.5355,  -7.2574,  -6.3438,  -8.3131,  -9.9237,
         -8.0688,  -7.6758,  -8.7705, -10.4571,  -9.6726,  -9.9571,  -7.3799,
         -8.5467, -11.4938,  -6.1687,  -8.4237, -11.8567, -10.2552,  -6.8779,
         -7.7484,  -9.6371,  -8.2765,  -9.0310,  -8.6967,  -8.8948,  -7.0017,
         -8.5431,  -8.0583,  -8.6952, -10.0346, -11.8954,  -8.1456,  -8.26

...

  0%|          | 200/125000 [00:13<2:18:32, 15.01it/s]

torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([5, 16, 17]) torch.Size([5, 16, 6]) torch.Size([5, 16]) torch.Size([5, 16]) torch.Size([5, 16])
tensor([ -8.6336, -12.0949,  -6.1650,  -9.1085,  -6.6822,  -7.6301,  -7.1829,
         -7.9360,  -6.3730,  -7.3375,  -6.4285,  -7.5316,  -8.5917, -10.4233,
         -7.7467,  -7.6158, -10.7483,  -9.1334, -11.6833,  -8.3276,  -7.7624,
         -9.7829,  -7.7504,  -9.8759, -10.4687,  -8.3048,  -6.1634,  -7.4640,
        -12.0488,  -6.9358,  -9.8225,  -9.9768,  -8.4069,  -7.1877,  -7.7831,
        -10.0579,  -7.1669,  -8.0042,  -8.7971,  -8.5306,  -8.0902, -13.4417,
         -8.3124,  -7.7272,  -7.6430,  -8.5147, -10.9564,  -8.6335,  -9.3009,
         -9.3439, -10.1234,  -8.0412,  -6.1148,  -7.8834, -10.3918,  -8.2262,
         -9.3374,  -8.2326,  -9.1151,  -8.5416,  -6.6018,  -6.6479,  -9.67

  0%|          | 202/125000 [00:13<2:16:15, 15.26it/s]


torch.Size([16, 6])
torch.Size([1, 6])
torch.Size([5, 16, 17]) torch.Size([5, 16, 6]) torch.Size([5, 16]) torch.Size([5, 16]) torch.Size([5, 16])
tensor([ -8.8133,  -8.8690, -10.7052,  -7.8553,  -7.9867, -12.6453, -11.0884,
         -8.0597, -14.0783,  -8.9305, -12.2697,  -8.3338,  -7.9580,  -7.1213,
         -6.1177, -10.2597,  -7.8068,  -8.4635,  -7.0204, -12.8072,  -6.6489,
        -11.0278,  -7.7099, -11.5533,  -7.8390,  -6.7872,  -7.6721,  -9.0330,
         -8.2619,  -7.7314, -13.0279,  -6.5950, -12.5153,  -9.0618,  -7.0773,
         -8.0243,  -9.5197,  -8.2567, -10.2596,  -8.4986,  -9.4049,  -6.8511,
         -9.6313, -10.5266,  -6.3229,  -7.6154, -10.4019,  -9.2263,  -7.9004,
         -9.0895,  -7.9749,  -7.8254,  -8.1590,  -8.1087,  -8.7948, -11.4508,
        -10.0966,  -6.7778,  -5.7149, -10.6557,  -7.8548,  -8.3381,  -8.5026,
         -8.1725,  -9.3994,  -6.0307,  -7.4261,  -7.8728, -10.2089,  -6.5280,
         -8.6227, -11.3244,  -8.7865,  -8.6625,  -7.6596,  -9.1468,  -9.19

KeyboardInterrupt: 

In [17]:
rundir_to_eval = Path(f"runs/{run_name}/2024_01_14_20_44_28")

In [19]:
print(f"Evaluating and capturing videos on {args.env_id}.")
mean_eval_return = eval_and_render(args=args, run_dir=rundir_to_eval)
print(f"Evaluation - Mean returns achieved: {mean_eval_return}.")

Evaluating and capturing videos on HalfCheetah-v4.
reading runs\A2C_PyTorch\2024_01_14_20_44_28/policy.pt...


c:\Users\nkvch\studia\USD\venv\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\nkvch\studia\USD\runs\A2C_PyTorch\2024_01_14_20_44_28\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


-> Episode 1: 715.8059692382812 returns
-> Episode 2: 1270.4517822265625 returns


KeyboardInterrupt: 

In [15]:
from IPython.display import Video

Video("half-cheetah-video.mp4")

ValueError: To embed videos, you must pass embed=True (this may make your notebook files huge)
Consider passing Video(url='...')